In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation, GaussianDropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy, Huber
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
#import tensorflow as tf
#import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices

from tensorflow.keras.constraints import max_norm, min_max_norm
from keras.callbacks import ReduceLROnPlateau

SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

#train = pd.read_csv('../input/jane-street-market-prediction/train.csv', nrows=1000)
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')


In [ ]:
train_part = train
train_part_mean = train_part.mean()
train_part_std = train_part.std()
train_part_var = train_part.var()
train_part.fillna(train_part_mean,inplace=True)

features = ['weight', 'feature_0', 'feature_1', 'feature_2', 'feature_3','feature_4', 
'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 
'feature_11', 'feature_12', 'feature_13',
'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18',
'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23',
'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28',
'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33',
'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38',
'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43',
'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48',
'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53',
'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58',
'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63',
'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68',
'feature_69', 'feature_70', 'feature_71', 'feature_72', 'feature_73',
'feature_74', 'feature_75', 'feature_76', 'feature_77', 'feature_78',
'feature_79', 'feature_80', 'feature_81', 'feature_82', 'feature_83',
'feature_84', 'feature_85', 'feature_86', 'feature_87', 'feature_88',
'feature_89', 'feature_90', 'feature_91', 'feature_92', 'feature_93',
'feature_94', 'feature_95', 'feature_96', 'feature_97', 'feature_98',
'feature_99', 'feature_100', 'feature_101', 'feature_102',
'feature_103', 'feature_104', 'feature_105', 'feature_106',
'feature_107', 'feature_108', 'feature_109', 'feature_110',
'feature_111', 'feature_112', 'feature_113', 'feature_114',
'feature_115', 'feature_116', 'feature_117', 'feature_118',
'feature_119', 'feature_120', 'feature_121', 'feature_122',
'feature_123', 'feature_124', 'feature_125', 'feature_126',
'feature_127', 'feature_128', 'feature_129', 'date','ts_id',
'resp_1','resp_2','resp_3','resp_4','resp']

train_part = train_part[features]
del(train)

In [ ]:
train_part = tf.convert_to_tensor(
    train_part, dtype="float32", dtype_hint=None, name=None
)
normalized_data_test=train_part # notactually normalized, but lets see if this makes life easier.



In [ ]:
train_part

In [ ]:
### CHANGE THIS BELOW for it to work based on dims of input....
dim_shape = 138 # this works 

# Creating a custom layer with keras API.
output_dim = 138
#output_dim = 138 #not sure which one to do
reg = 0.01

def Coupling(input_shape):
    input = keras.layers.Input(shape=input_shape)

    t_layer_1 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(input)
    t_layer_2 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_1)
    t_layer_3 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_2)
    t_layer_4 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_3)
    t_layer_5 = keras.layers.Dense(
        input_shape, activation="linear", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_4)

    s_layer_1 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(input)
    s_layer_2 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_1)
    s_layer_3 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_2)
    s_layer_4 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_3)
    s_layer_5 = keras.layers.Dense(
        input_shape, activation="tanh", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_4)

    return keras.Model(inputs=input, outputs=[s_layer_5, t_layer_5])

class RealNVP(keras.Model):
    def __init__(self, num_coupling_layers):
        super(RealNVP, self).__init__()

        self.num_coupling_layers = num_coupling_layers

        # Distribution of the latent space.
#        self.distribution = tfp.distributions.MultivariateNormalDiag(
#            loc=[0.0, 0.0], scale_diag=[1.0, 1.0]
#        )
        self.distribution = tfp.distributions.MultivariateNormalDiag(
            loc=np.zeros(dim_shape).tolist(), scale_diag=np.ones(dim_shape).tolist()
#            loc=np.zeros(dim_shape).tolist(), scale_diag=np.ones(3).tolist()
#            loc=np.zeros(3).tolist(), scale_diag=np.ones(3).tolist()
        )

#        self.masks = np.array(
#            [[0, 1], [1, 0]] * (num_coupling_layers // 2), dtype="float32"
#        )
        # need mto make this dynamic e.g. identiy(3) will be euiqvalen to size ofrectangle
    # also need to check this against original moons data with size of 2 to see if it works..
        self.masks = np.array(
            np.identity(dim_shape).tolist() * (num_coupling_layers // 2), dtype="float32"
#            np.identity(3).tolist() * (num_coupling_layers // 2), dtype="float32"
        )
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.layers_list = [Coupling(dim_shape) for i in range(num_coupling_layers)]
#        self.layers_list = [Coupling(3) for i in range(num_coupling_layers)]

    @property
    def metrics(self):
        """List of the model's metrics.
        We make sure the loss tracker is listed as part of `model.metrics`
        so that `fit()` and `evaluate()` are able to `reset()` the loss tracker
        at the start of each epoch and at the start of an `evaluate()` call.
        """
        return [self.loss_tracker]

    def call(self, x, training=True):
        log_det_inv = 0
        direction = 1
        if training:
            direction = -1
        for i in range(self.num_coupling_layers)[::direction]:
#            import pdb;pdb.set_trace()
            x_masked = x * self.masks[i]
            x_masked = x * self.masks[i]
            reversed_mask = 1 - self.masks[i]
#            import pdb;pdb.set_trace()
            s, t = self.layers_list[i](x_masked)
            s *= reversed_mask
            t *= reversed_mask
            gate = (direction - 1) / 2
            x = (
                reversed_mask
                * (x * tf.exp(direction * s) + direction * t * tf.exp(gate * s))
                + x_masked
            )
            log_det_inv += gate * tf.reduce_sum(s, [1])

        return x, log_det_inv

    # Log likelihood of the normal distribution plus the log determinant of the jacobian.

    def log_loss(self, x):
        y, logdet = self(x)
#        import pdb;pdb.set_trace()
        log_likelihood = self.distribution.log_prob(y) + logdet
        return -tf.reduce_mean(log_likelihood)

    def train_step(self, data):
        with tf.GradientTape() as tape:

            loss = self.log_loss(data)

        g = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(g, self.trainable_variables))
        self.loss_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self.log_loss(data)
        self.loss_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result()}

    


In [ ]:
import os
checkpoint_path = "training_PRODUCTION/cp_PRODUCTION.ckpt"
checkpoint_path_read = "training_PRODUCTION/cp_PRODUCTION.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
try:
    !mkdir training_PRODUCTION/
except:
    print('couldnt make directory')

try:
    !cp /kaggle/input/nvp-to-mlp-impute-sgdmultidraw-fixrespaction/training_PRODUCTION/cp_PRODUCTION.ckpt.index training_PRODUCTION/cp_PRODUCTION.ckpt.index
    !cp /kaggle/input/nvp-to-mlp-impute-sgdmultidraw-fixrespaction/training_PRODUCTION/cp_PRODUCTION.ckpt.data-00000-of-00001 training_PRODUCTION/cp_PRODUCTION.ckpt.data-00000-of-00001
    !cp /kaggle/input/nvp-to-mlp-impute-sgdmultidraw-fixrespaction/training_PRODUCTION/cp_PRODUCTION.ckpt.index training_PRODUCTION/cp_PRODUCTION.ckpt.index

except:
    print('error in copying over.')

In [ ]:

model_chk_latent = RealNVP(num_coupling_layers=6)
model_chk_latent.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001))
# 3 minutes to evaluate
model_chk_latent.evaluate(normalized_data_test)
try:
    # you had to add the data source first from the notebook, not you can read in this cross-notebook data..
    model_chk_latent.load_weights(checkpoint_path_read)

    # 3 minutes to evaluate
    model_chk_latent.evaluate(normalized_data_test)
#    model_chk_latent = model_chk

    model_chk_latent.fit(
    normalized_data_test, batch_size=256, epochs=15, verbose=2, validation_split=0.5, callbacks = [cp_callback]
    )
except:
    print('fitting for the first time...')

    model_chk_latent.fit(
    normalized_data_test, batch_size=256, epochs=1, verbose=2, validation_split=0.5, callbacks = [cp_callback]
    )

In [ ]:
#normalized_data_part = normalized_data_test[1500000:2390491]
#normalized_data_part = .sample(1)

## if you want to plot uncomment the below. 
## but if just generating data. You don't need to do the first piece.
#print(normalized_data_part)
# From data to latent space.
#start=1200000
#end=2390491
#z, _ = model_chk(normalized_data_test[start:end])
#print(z)

#From latent space to data.
#samples_1 = model_chk.distribution.sample(500000)
#samples_2 = model_chk.distribution.sample(500000)
#samples_3 = model_chk.distribution.sample(500000)
#samples_4 = model_chk.distribution.sample(500000)
#samples_5 = model_chk.distribution.sample(500000)
#samples = model_chk.distribution.sample(3000)
#x, _ = model_chk.predict(samples)


In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation, GaussianDropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy, Huber
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices

from tensorflow.keras.constraints import max_norm, min_max_norm
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, LeakyReLU

# Add the GELU function to Keras
#https://mlfromscratch.com/activation-functions-explained/#/
    
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(x * 0.7978845608 * (1 + 0.044715 * x * x)))
    #https://github.com/hendrycks/GELUs
    #exact version is better not sure if below if exact and above is approximation
    
# not sure if below is considered exact
#    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

# Add leaky-relu so we can use it as a string
get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

act_func = ['sigmoid', 'relu', 'elu', 'leaky-relu', 'selu', 'gelu']

In [ ]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    x = tf.keras.layers.GaussianDropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        # tf.keras.initializers.he_normal
        # tf.keras.initializers.he_uniform (both of these are said to be good for a relu family of activations)
#        x = tf.keras.layers.Dense(hidden_units[i],kernel_initializer=tf.keras.initializers.he_normal(seed=1111))(x)
#        x = tf.keras.layers.Dense(hidden_units[i],kernel_initializer=tf.keras.initializers.he_uniform(seed=1111))(x)
#        x = tf.keras.layers.Dense(hidden_units[i],kernel_initializer=tf.keras.initializers.Orthogonal(seed=1111))(x)
#        x = tf.keras.layers.Dense(hidden_units[i],kernel_initializer=tf.keras.initializers.VarianceScaling(seed=1111))(x)
#        x = tf.keras.layers.Dense(hidden_units[i],kernel_initializer=tf.keras.initializers.TruncatedNormal(seed=1111))(x)
#        x = tf.keras.layers.Dense(hidden_units[i], kernel_constraint=min_max_norm(min_value=-0.5, max_value=0.5, rate=1.0, axis=0))(x)
#        x = tf.keras.layers.Dense(hidden_units[i],
#                                  kernel_initializer=tf.keras.initializers.TruncatedNormal(seed=1111),
#                                  kernel_constraint=min_max_norm(min_value=-0.5, max_value=0.5, rate=1.0, axis=0))(x)
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
#        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Activation('gelu')(x)
#        x = tf.keras.layers.Activation('relu')(x)
# introduce randomness e.g. flip a coin and choose different dropouts
#        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
        x = tf.keras.layers.GaussianDropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    # i would like to try something else then just sigmoid 
    # try output of linear layer, then input into softmax instead....
    # or then input into sigmoid as from logits
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
#        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
# introduce randomness e.g. flip a coin and choose difference losses
#        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        loss=tf.keras.losses.Huber(),
#        metrics=tf.keras.metrics.AUC(name="AUC")
        metrics=[tf.keras.metrics.AUC(name="AUC"),
                 tf.keras.metrics.TrueNegatives(name="TN"),
                 tf.keras.metrics.TruePositives(name="TP")]
#                 tf.keras.metrics.SpecificityAtSensitivity(name="SpeciSensi")]
        # many other metrics you could try...
    )

    return model

In [ ]:
try:
    !mkdir training_MLP/
except:
    print('couldnt make directory')

try:
# issues with recursion previously, so going to just copy what is necessary...
#    !cp -r /kaggle/input/keras-normalizing-flow-backup/training_PRODUCTION/ training_PRODUCTION/
    !cp /kaggle/input/nvp-to-mlp-impute-sgdmultidraw-fixrespaction/training_MLP/cp_PRODUCTION.ckpt.index training_MLP/cp_PRODUCTION.ckpt.index
    !cp /kaggle/input/nvp-to-mlp-impute-sgdmultidraw-fixrespaction/training_MLP/cp_PRODUCTION.ckpt.data-00000-of-00001 training_MLP/cp_PRODUCTION.ckpt.data-00000-of-00001
    !cp /kaggle/input/nvp-to-mlp-impute-sgdmultidraw-fixrespaction/training_MLP/cp_PRODUCTION.ckpt.index training_MLP/cp_PRODUCTION.ckpt.index

except:
    print('error in copying over.')

In [ ]:
checkpoint_path = "training_MLP/cp_PRODUCTION.ckpt"
checkpoint_path_read = "training_MLP/cp_PRODUCTION.ckpt"
#checkpoint_path_read = "/kaggle/input/nvp-to-mlp-impute-sgdmultidraw/training_MLP/cp_PRODUCTION.ckpt"
#checkpoint_path_read_fulldir = "/kaggle/input/nvp-to-mlp-impute-sgdmultidraw/training_MLP/"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

iter_over_sgd = 2
for i in range(iter_over_sgd):
    print(i)
    samples = model_chk_latent.distribution.sample(500000)
    samples_np = samples.numpy() # shouldn't do it this way, but lets first start by making sure no issues in data syncing between two models
    print(samples_np.mean())
    del(samples)
    train = pd.DataFrame(samples_np,columns=features)
    
    x_features=train.columns[0:133]
    x_features_min=train.columns[0:132]
    X_train=train.loc[:,x_features]

    y_features = train.columns[133:]
    y_train = np.stack([(train[c]) for c in y_features]).T

    batch_size = 5000
    hidden_units = [260, 260, 260, 260, 260, 260]
    #dropout_rates = [0.2, 0.2, 0.2, 0.2, 0.5, 0.2, 0.2]
    dropout_rates = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
    #label_smoothing = 1e-2
    learning_rate = 1e-2
#    learning_rate = 1e-3
    #learning_rate = 1e-4

    model_chk = create_mlp(len(x_features), 5, hidden_units, dropout_rates, learning_rate)
    # 3 minutes to evaluate
    #model_chk.evaluate(normalized_data_test)
    model_chk.evaluate(X_train, y_train)
    try:
        model_chk.load_weights(checkpoint_path_read)
        # 3 minutes to evaluate
        model_chk.evaluate(X_train, y_train)
        model_chk.fit(X_train, y_train, epochs=25, batch_size=5000, callbacks=[cp_callback])
    except:
        print('failed going to refit the model, careful as you can lose all progress')
        model_chk.fit(X_train, y_train, epochs=1, batch_size=5000, callbacks=[cp_callback])

    

In [ ]:
#submission = True
submission = False

if(submission==False):
    import janestreet
    janestreet.make_env.__called__ = False

In [ ]:
import numpy as np
from numba import njit

#@njit
#def fillna_npwhere_njit(array, values):
#    if np.isnan(array.sum()):
#        array = np.where(np.isnan(array), values, array)
#    return array

f = np.median
import janestreet
env = janestreet.make_env()
# if calculating a utility score during training, should have way to store this
if(submission==False):
    store_data_1 = []
    store_data_2 = []
    store_data_3 = []
    test_df_arr = []
    pred_df_arr = []
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
#    if test_df['weight'].item() > 3.089004755: # mean with action > mean in sample
#    if test_df['weight'].item() > 2.52845072: # mean of sample
#    if test_df['weight'].item() > 2.42904520: # mean with action > mean 0.52 in sample
#    if test_df['weight'].item() > 1.75373077: # median with action > mean 0.52 in sample
#        import pdb;pdb.set_trace()
        ts_id = 0 # make this keep track later # e.g. if date > previous date, then ts_id = 0 else ts_id +=1
#        x_tt = np.append(test_df.loc[:, x_features_min].values,np.array([ts_id,0,0,0,0,0]))
        x_tt = np.append(test_df.loc[:, x_features_min].values,np.array([ts_id,999,999,999,999,999]))
#        x_tt = np.append(test_df.loc[:, x_features_min].values,np.array([ts_id,-999,-999,-999,-999,-999]))
        if np.isnan(x_tt.sum()):
            x_tt = np.where(np.isnan(x_tt), 0, x_tt)
        x_tt_latent = model_chk_latent([x_tt], training=False)[0].numpy()[0] 
        ## what happens if i just predict from latent?
        pred = np.where(np.mean(x_tt_latent[133:]) >= 0.50, 1, 0).astype(int)
        pred_df.action = pred
#        tensor_x_tt_latent_reorg = tf.convert_to_tensor([x_tt_latent[0:133]], dtype="float32", dtype_hint=None, name=None)
#        pred = np.mean(model_chk(tensor_x_tt_latent_reorg, training = False).numpy())
#        pred_df.action = np.where(pred >= 0.50, 1, 0).astype(int)
        
    else:
        pred_df.action = 0
        pred=0.0
    env.predict(pred_df)
    if(submission==False):
        store_data_1.append(pred_df)
        store_data_2.append(test_df)
        store_data_3.append(pred)
        test_df_arr.append(test_df)
        pred_df_arr.append(pred_df)
        
# last speed was 15219it [13:25, 18.88it/s]
# 15219it [13:12, 19.21it/s]
# 15219it [11:57, 21.21it/s]

In [ ]:
#if this looks good...
# maybe i should just figure out how to serve the model faster
# eg via tfxlite or quantization pieces???

In [ ]:
#%timeit model_chk_latent([fillna_npwhere_njit(np.append(test_df.loc[:, x_features_min].values,np.array([ts_id,0,0,0,0,0])), 0)], training=False)[0].numpy()[0]

In [ ]:
#%timeit np.mean(model_chk(tensor_x_tt_latent_reorg, training = False).numpy())

In [ ]:
#%timeit model_chk_latent([x_tt], training=False)[0].numpy()[0]

In [ ]:
#%timeit fillna_npwhere_njit(x_tt, 0)

In [ ]:
#%timeit tf.convert_to_tensor([x_tt_latent[0:133]], dtype="float32", dtype_hint=None, name=None)

In [ ]:
#%timeit np.append(test_df.loc[:, x_features_min].values,np.array([ts_id,0,0,0,0,0]))

In [ ]:
#%timeit np.where(pred >= 0.50, 1, 0).astype(int)#

In [ ]:
if(submission==False):
    df_data_1 = pd.concat(store_data_1)
    df_data_2 = pd.concat(store_data_2)
    df_data_2['action'] = df_data_1['action']
    df_data_2['resp'] = store_data_3 #careful as some values are just repeat of last value sicne skipped, but we are filterin gthese out
    train_del = df_data_2
    print(df_data_2.weight.mean())
    
    features_del = [c for c in train_del.columns if 'feature' in c]
    print('Forward-Filling...')
    train_del = train_del.query('weight > 0').reset_index(drop = True)
    train_del[features_del] = train_del[features_del].fillna(method = 'ffill').fillna(0)

    date = train_del['date'].values #* 1.0
    weight = train_del['weight'].values #* 1.0
    resp = train_del['resp'].values #* 1.0 # just doing this for now to debug... not sure if my 1 and 0 will also cause issues
    action = train_del['action'].values #* 1.0


    date = date.astype(np.int64)
    weight = weight.astype(np.float64)
    resp = resp.astype(np.float64)
    action = action.astype(np.int64)
    
    from numba import njit
    @njit(fastmath = True)
    def utility_score_numba(date, weight, resp, action):
    #    import pdb;pdb.set_trace()
        Pi = np.bincount(date, weight * resp * action)
        t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
        u = min(max(t, 0), 6) * np.sum(Pi)
        return u
    
    somevalue = utility_score_numba(date, weight, resp, action)
    print(somevalue)
    assumed_number = 50
    print(somevalue/assumed_number)

In [ ]:
#2.5032816647355256
#Forward-Filling...
#46239.871442783515
#924.7974288556703

#2.5032816647355256
#Forward-Filling...
#46239.871442783515
#924.7974288556703

In [ ]:
#samples = model_chk_latent.distribution.sample(500000)
#samples_np = samples.numpy() # shouldn't do it this way, but lets first start by making sure no issues in data syncing between two models
#print(samples_np.mean())
#x, _ = model_chk.predict(samples)
#del(samples)
#train = pd.DataFrame(samples_np,columns=features)
#train_nonlatent = pd.DataFrame(x,columns=features)


In [ ]:
# detect and init the TPU
#import tensorflow as tf
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

#batch_size = 5000
#hidden_units = [130, 130, 130, 130, 130, 130]
#dropout_rates = [0.2, 0.2, 0.2, 0.2, 0.5, 0.2, 0.2]
#dropout_rates = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
#label_smoothing = 1e-2
#learning_rate = 1e-2
#learning_rate = 1e-3
#learning_rate = 1e-4

# instantiating the model in the strategy scope creates the model on the TPU
#with tpu_strategy.scope():
#    model = tf.keras.Sequential( … ) # define your model normally
#    model.compile( … )
#    clf = create_mlp(
#        len(features), 5, hidden_units, dropout_rates, learning_rate
#        )


# train model normally
#model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.001)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=10, min_lr=0.001)
#clf.fit(X_train, y_train, epochs=5, batch_size=5000, callbacks=[reduce_lr])
#clf.fit(X_train, y_train, epochs=800, batch_size=5000, callbacks=[reduce_lr])
#clf.fit(X_train, y_train, epochs=800, batch_size=5000) # try reduce_lr next


#clf = create_mlp(len(features), 5, hidden_units, dropout_rates, learning_rate)

#clf.fit(X_train, y_train, epochs=5, batch_size=5000)
#clf.fit(X_train, y_train, epochs=800, batch_size=5000)
#clf.fit(X_train, y_train, epochs=300, batch_size=5000)
#clf.fit(X_train, y_train, epochs=300, batch_size=5000)
#clf.fit(X_train, y_train, epochs=200, batch_size=5000)


#models = []

#models.append(clf)

#th = 0.5000

In [ ]:
#### maybe need to investigate quantization vs normalization...
#### mabe quantization would hav been better option come prediction time, to have down also during generative modelling...
